# Análise dos preços de apartamentos em Salvador

A análise será feita com os dados extraídos na OLX em 04/02/2023.

### Extração dos dados no site da OLX

In [238]:
import requests
from bs4 import BeautifulSoup

page_olx = 1
URL = "https://www.olx.com.br/imoveis/venda/estado-ba/grande-salvador/salvador?o={page_olx}"

headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',
    'accept-language':'pt-BR,pt;q=0.6',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding':'gzip, deflate'
}




In [239]:
response = requests.get(url=URL,headers=headers)
salvador_olx_data = response.text
soup = BeautifulSoup(salvador_olx_data, "html.parser")

In [240]:
#Lista de itens na página
page_data_items = soup.select("#ad-list > li")
item = page_data_items[1]

#Título do anuncio:
titulo = item.select("a > div.sc-fgfRvd.fZoIzL > div.sc-hIVACf.dUnJVd > div.sc-gpHHfC.kJrkKt > h2")
titulo_anuncio = titulo[0].text

#Tipo de imóvel:
tp_imovel_item = item.select("h3")
tp_imovel_item = [x for x in tp_imovel_item if "venda" in x.text]
tp_imovel = tp_imovel_item[0].text

#Informações que não existem em todos os anuncios
quarto_info, area_info, garagem_info, banheiro_info, iptu_info, condominio_info = '','','','','',''

#Informações de banheiro, area, quartos e garagem:
for i in range(1,8):
    try:
        atributo = item.select(f"ul > li:nth-child({i}) > span")[0].attrs['aria-label']
        if 'QUARTO' in atributo.upper():
            quarto_info = atributo
        elif 'METRO' in atributo.upper():
            area_info = atributo
        elif 'GARAGEM' in atributo.upper():
            garagem_info = atributo
        elif 'BANHEIRO' in atributo.upper():
            banheiro_info = atributo 
    except:
        pass

#Dados de localização:
localizacao_item = item.select('a > div.sc-fgfRvd.fZoIzL > div.sc-kQsIoO.kbWREw > div > div.sc-hjRWVT.lgwjMd > span:nth-child(2)')[0]
cidade, bairro = [x.strip() for x in localizacao_item.text.split(',')]

#Valores:
preco_item = item.select('a > div.sc-fgfRvd.fZoIzL > div.sc-hIVACf.dUnJVd > div.sc-cpmKsF.ecSOri > span.main-price')
preco_imovel = preco_item[0].text

#Condomínio e IPTU:
for i in range(1,4):
    try:
        valor_atributo = item.select(f'a > div.sc-fgfRvd.fZoIzL > div.sc-hIVACf.dUnJVd > div.sc-cpmKsF.ecSOri > span:nth-child({i})')[0].text
        if 'IPTU' in valor_atributo.upper():
            iptu_info = valor_atributo
        elif 'CONDOM' in valor_atributo.upper():
            condominio_info = valor_atributo
    except:
        pass

#Link do anúncio:
link_anuncio = item.select('a')[0].attrs['href']


#Data do anúncio:
data_anuncio_info = item.select('a > div.sc-fgfRvd.fZoIzL > div.sc-kQsIoO.kbWREw > div > div.sc-hjRWVT.lgwjMd > span:nth-child(4)')[0].text


In [241]:
print(f'{titulo_anuncio}\n{tp_imovel}\n{quarto_info}\n{area_info}\n{garagem_info}\n{banheiro_info}\n{cidade}\n{bairro}\n{preco_imovel}\n{iptu_info}\n{condominio_info}\n{link_anuncio}\n{data_anuncio_info}')


PITUBA APARTAMENTO 1/4 AMPLO 54 M2 GARAGEM EXCELENTE LOCALIZAÇÃO
Apartamento À venda
1 quarto
54 metros quadrados
1 vaga de garagem
1 banheiro
Salvador
Pituba
R$ 235.030
IPTU R$ 1
Condomínio R$ 690
https://ba.olx.com.br/grande-salvador/imoveis/pituba-apartamento-1-4-amplo-54-m2-garagem-excelente-localizacao-1142463603
3 de fev, 16:18


Exemplos das informações acima:

Greenville  Etco 3/4 com varanda gourmet, andar alto - PATAMARES

Apartamento À venda

3 quartos

130 metros quadrados

2 vagas de garagem

3 banheiros

Salvador

Patamares

R$ 1.230.000

IPTU R$ 330

Condomínio R$ 800

https://ba.olx.com.br/grande-salvador/imoveis/greenville-etco-3-4-com-varanda-gourmet-andar-alto-patamares-1143011596

Ontem, 23:58 ou 3 de fev, 16:18